In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_test = pd.read_csv('./data/data_test.csv')
data_train = pd.read_csv('./data/data_train.csv')

In [3]:
data_train.head()

,Unnamed: 0,id,vas_id,buy_time,target
0,0,540968,8.0,1537131600,0.0
1,1,1454121,4.0,1531688400,0.0
2,2,2458816,1.0,1534107600,0.0
3,3,3535012,5.0,1535922000,0.0
4,4,1693214,1.0,1535922000,0.0


In [4]:
data_test.head()

,Unnamed: 0,id,vas_id,buy_time
0,0,3130519,2.0,1548018000
1,1,2000860,4.0,1548018000
2,2,1099444,2.0,1546808400
3,3,1343255,5.0,1547413200
4,4,1277040,2.0,1546808400


In [5]:
import dask.dataframe as dd
features = dd.read_csv('./data/features.csv', blocksize=25e6, sep='\t')

In [6]:
# features которые есть в data_train и в data_test
features_train = features.loc[features['id'].isin(data_train['id'])].compute()
# features_test = features.loc[features['id'].isin(data_test['id'])].compute()

### Обзор data_train 

In [7]:
data_train['id'].shape

(831653,)

In [8]:
data_train['id'].nunique()

806613

In [9]:
non_unique = data_train['id'].shape[0] - data_train['id'].nunique()
non_unique

25040

In [10]:
data_train[data_train['id'].duplicated(keep=False)].sort_values(by='id').head()

,Unnamed: 0,id,vas_id,buy_time,target
14042,14042,342,2.0,1542574800,1.0
14041,14041,342,1.0,1541365200,0.0
19825,19825,492,6.0,1544994000,1.0
19824,19824,492,4.0,1538946000,0.0
22908,22908,579,4.0,1542574800,1.0


Один и тот же пользователб может в разные моменты времени покупать и отказываться от разных услуг. Нужно использовать соответствующий профиль пользователя из featuresна момент поекпки или отказа.

### Обзор features_train

In [11]:
features_train.duplicated().sum()

0

In [12]:
features_train.shape

(834168, 256)

In [13]:
features_train['id'].nunique()

806613

In [14]:
non_unique = features_train.shape[0] - features_train['id'].nunique()
non_unique

27555

In [15]:
features_train[features_train['id'].duplicated(keep=False)].sort_values(by='id').head()

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
3476,3664458,372,1544389200,159.560029,-100.259112,146.889214,-110.936798,141.471754,-132.04179,-16.08618,...,4405.626154,1933.229208,-25.996269,-37.630448,29.252276,-22.832889,-0.694428,-12.175933,-0.45614,0.0
893,982668,372,1544994000,-96.799971,-143.929112,-109.050786,-174.096798,-114.468246,-195.20179,-16.08618,...,-835.373846,-471.770792,-25.996269,-37.630448,-290.747724,-22.832889,-0.694428,-12.175933,-0.45614,0.0
3156,914805,404,1533502800,-78.159971,499.180888,-84.480786,460.553202,-89.898246,439.44821,-16.08618,...,-408.373846,-588.770792,-25.996269,-22.630448,861.252276,34.167111,-0.694428,47.824067,-0.45614,0.0
4910,3600789,404,1540155600,354.550029,164.940888,526.199214,327.583202,520.781754,306.47821,403.40382,...,2712.626154,1326.229208,-24.996269,102.369552,201.252276,30.167111,-0.694428,-10.175933,-0.45614,0.0
784,2890548,487,1544389200,-28.169971,-30.359112,119.749214,417.873202,114.331754,396.76821,-16.08618,...,-973.373846,-613.770792,-25.996269,-9.630448,11.252276,45.167111,0.305572,-12.175933,2.54386,1.0


In [16]:
dublicated = features_train[features_train['id'].duplicated(keep=False)].sort_values(by='id')['id'].value_counts()
dublicated

2101246    2
3403223    2
2797023    2
3429856    2
1859041    2
          ..
1653490    2
1782515    2
1706740    2
3762936    2
4003250    2
Name: id, Length: 27555, dtype: int64

### отдельная запись датасета

In [17]:
user = data_train.loc[data_train['id']==372]
user.head()

,Unnamed: 0,id,vas_id,buy_time,target
15681,15681,372,4.0,1535922000,0.0


In [18]:
user_feature = features_train.loc[features_train['id']==372]
user_feature.head()

,Unnamed: 0,id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
893,982668,372,1544994000,-96.799971,-143.929112,-109.050786,-174.096798,-114.468246,-195.20179,-16.08618,...,-835.373846,-471.770792,-25.996269,-37.630448,-290.747724,-22.832889,-0.694428,-12.175933,-0.45614,0.0
3476,3664458,372,1544389200,159.560029,-100.259112,146.889214,-110.936798,141.471754,-132.04179,-16.08618,...,4405.626154,1933.229208,-25.996269,-37.630448,29.252276,-22.832889,-0.694428,-12.175933,-0.45614,0.0


In [19]:
merged = pd.merge(user, user_feature, on='id')
merged.head()

,Unnamed: 0_x,id,vas_id,buy_time_x,target,Unnamed: 0_y,buy_time_y,0,1,2,...,243,244,245,246,247,248,249,250,251,252
0,15681,372,4.0,1535922000,0.0,982668,1544994000,-96.799971,-143.929112,-109.050786,...,-835.373846,-471.770792,-25.996269,-37.630448,-290.747724,-22.832889,-0.694428,-12.175933,-0.45614,0.0
1,15681,372,4.0,1535922000,0.0,3664458,1544389200,159.560029,-100.259112,146.889214,...,4405.626154,1933.229208,-25.996269,-37.630448,29.252276,-22.832889,-0.694428,-12.175933,-0.45614,0.0


### Merge data_train и features_train

In [20]:
data_merged_train = pd.merge(data_train, features_train, on='id')
data_merged_train.head()

,Unnamed: 0_x,id,vas_id,buy_time_x,target,Unnamed: 0_y,buy_time_y,0,1,2,...,243,244,245,246,247,248,249,250,251,252
0,0,540968,8.0,1537131600,0.0,3756522,1541970000,-31.559971,327.360888,-45.500786,...,-845.373846,-613.770792,-20.996269,-37.630448,-28.747724,4.167111,7.305572,-12.175933,21.54386,0.0
1,1,1454121,4.0,1531688400,0.0,2735973,1531083600,547.270029,238.430888,533.329214,...,-972.373846,-613.770792,-25.996269,-19.630448,-278.747724,-24.832889,-0.694428,-11.175933,-0.45614,0.0
2,2,2458816,1.0,1534107600,0.0,3586550,1543179600,-92.139971,-95.469112,-106.080786,...,-977.373846,-613.770792,-25.996269,-37.630448,-304.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,3,3535012,5.0,1535922000,0.0,1634974,1533502800,54.880029,12.970888,54.079214,...,-977.373846,-613.770792,-25.996269,-18.630448,-133.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0
4,4,1693214,1.0,1535922000,0.0,716515,1543179600,45.160029,295.240888,64.679214,...,-965.373846,-612.770792,-22.996269,-32.630448,-127.747724,-4.832889,-0.694428,-12.175933,-0.45614,0.0


In [21]:
tmp = data_merged_train.loc[data_merged_train['id'].isin(dublicated.index)]
tmp.head()

,Unnamed: 0_x,id,vas_id,buy_time_x,target,Unnamed: 0_y,buy_time_y,0,1,2,...,243,244,245,246,247,248,249,250,251,252
8,8,2000856,1.0,1534712400,0.0,2114914,1531688400,-21.369971,108.780888,178.619214,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
9,8,2000856,1.0,1534712400,0.0,4145047,1531083600,-92.559971,48.600888,50.239214,...,-977.373846,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
58,57,3577738,1.0,1532293200,0.0,2986290,1540760400,-96.799971,45.210888,-104.810786,...,-929.373846,-565.770792,-25.996269,-28.630448,-177.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
59,57,3577738,1.0,1532293200,0.0,3235297,1537736400,-96.799971,104.530888,-8.220786,...,-969.373846,-605.770792,-25.996269,-35.630448,-159.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
63,61,203194,1.0,1532293200,0.0,146332,1542574800,-96.799971,-111.569112,-110.740786,...,-977.373846,-613.770792,-25.996269,-37.630448,-258.747724,-24.832889,-0.694428,-12.175933,-0.45614,0.0


In [22]:
tmp['delta'] = abs(tmp['buy_time_x'] - tmp['buy_time_y'])
tmp.sort_values(by = ['Unnamed: 0_x', 'delta'], inplace=True)
tmp.head()

<ipython-input-22-51c25ed858b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['delta'] = abs(tmp['buy_time_x'] - tmp['buy_time_y'])
<ipython-input-22-51c25ed858b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.sort_values(by = ['Unnamed: 0_x', 'delta'], inplace=True)


,Unnamed: 0_x,id,vas_id,buy_time_x,target,Unnamed: 0_y,buy_time_y,0,1,2,...,244,245,246,247,248,249,250,251,252,delta
8,8,2000856,1.0,1534712400,0.0,2114914,1531688400,-21.369971,108.780888,178.619214,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,3024000
9,8,2000856,1.0,1534712400,0.0,4145047,1531083600,-92.559971,48.600888,50.239214,...,-613.770792,-25.996269,-37.630448,-306.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,3628800
59,57,3577738,1.0,1532293200,0.0,3235297,1537736400,-96.799971,104.530888,-8.220786,...,-605.770792,-25.996269,-35.630448,-159.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,5443200
58,57,3577738,1.0,1532293200,0.0,2986290,1540760400,-96.799971,45.210888,-104.810786,...,-565.770792,-25.996269,-28.630448,-177.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,8467200
64,61,203194,1.0,1532293200,0.0,2041758,1540760400,-96.799971,-170.889112,-110.740786,...,-613.770792,-25.996269,-37.630448,-297.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0,8467200


In [23]:
duplicates = tmp['Unnamed: 0_x'].duplicated()
duplicates_to_delete = duplicates[duplicates.values == True]
data_merged_train.drop(duplicates_to_delete.index, axis=0, inplace=True)
data_merged_train.head()

,Unnamed: 0_x,id,vas_id,buy_time_x,target,Unnamed: 0_y,buy_time_y,0,1,2,...,243,244,245,246,247,248,249,250,251,252
0,0,540968,8.0,1537131600,0.0,3756522,1541970000,-31.559971,327.360888,-45.500786,...,-845.373846,-613.770792,-20.996269,-37.630448,-28.747724,4.167111,7.305572,-12.175933,21.54386,0.0
1,1,1454121,4.0,1531688400,0.0,2735973,1531083600,547.270029,238.430888,533.329214,...,-972.373846,-613.770792,-25.996269,-19.630448,-278.747724,-24.832889,-0.694428,-11.175933,-0.45614,0.0
2,2,2458816,1.0,1534107600,0.0,3586550,1543179600,-92.139971,-95.469112,-106.080786,...,-977.373846,-613.770792,-25.996269,-37.630448,-304.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,3,3535012,5.0,1535922000,0.0,1634974,1533502800,54.880029,12.970888,54.079214,...,-977.373846,-613.770792,-25.996269,-18.630448,-133.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0
4,4,1693214,1.0,1535922000,0.0,716515,1543179600,45.160029,295.240888,64.679214,...,-965.373846,-612.770792,-22.996269,-32.630448,-127.747724,-4.832889,-0.694428,-12.175933,-0.45614,0.0


In [24]:
data_merged_train.shape

(831653, 260)

Таким образом количество строк теперь равно количеству строк в data_tarin. Если для данного пользователя в features_train было несколько записей, учитываем только ту, которая по времени была ближе к той, которая в data_train

In [25]:
data_merged_train.drop(['Unnamed: 0_x', 'Unnamed: 0_y', 'buy_time_y'], axis=1, inplace=True)
data_merged_train.head()

,id,vas_id,buy_time_x,target,0,1,2,3,4,5,...,243,244,245,246,247,248,249,250,251,252
0,540968,8.0,1537131600,0.0,-31.559971,327.360888,-45.500786,274.753202,-50.918246,253.64821,...,-845.373846,-613.770792,-20.996269,-37.630448,-28.747724,4.167111,7.305572,-12.175933,21.54386,0.0
1,1454121,4.0,1531688400,0.0,547.270029,238.430888,533.329214,274.803202,527.911754,253.69821,...,-972.373846,-613.770792,-25.996269,-19.630448,-278.747724,-24.832889,-0.694428,-11.175933,-0.45614,0.0
2,2458816,1.0,1534107600,0.0,-92.139971,-95.469112,-106.080786,-139.596798,-111.498246,-142.90179,...,-977.373846,-613.770792,-25.996269,-37.630448,-304.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,3535012,5.0,1535922000,0.0,54.880029,12.970888,54.079214,-9.116798,48.661754,-30.22179,...,-977.373846,-613.770792,-25.996269,-18.630448,-133.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0
4,1693214,1.0,1535922000,0.0,45.160029,295.240888,64.679214,344.283202,59.261754,323.17821,...,-965.373846,-612.770792,-22.996269,-32.630448,-127.747724,-4.832889,-0.694428,-12.175933,-0.45614,0.0


In [26]:
data_merged_train.to_csv('./data/my_train.csv')